<a href="https://colab.research.google.com/github/FadelYang/craft-it-object-detection-model/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing required Libraries
import os
import tensorflow as tf
import numpy as np
import cv2

# Preparing Dataset

## Mount to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/'Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


## Connect To Kaggle And Download Datasets

this code is used to connect Google Colab and Kaggle, run it just once for download the datasets from kaggle, uncomment it first if you want to run these cell below

In [4]:
# # copy kaggle.json to /root/.kaggle/ folder so that kaggle cli can access it.
# !mkdir /.kaggle
# !mv kaggle.json /.kaggle
# !mv /.kaggle /root/
# !chmod 600 ~/.kaggle/kaggle.json

In [5]:
# # Download datasets from Kaggle.
# !kaggle datasets download -d kshitizgajurel042/recyclable-images-with-annotations-for-detection

In [6]:
# !unzip recyclable-images-with-annotations-for-detection.zip

In [7]:
# !mv 'Final Data' recyclable-images-datasets

## Preprocess Dataset

In [8]:
# Define function for read annotations
def read_annotations(annotation_file):
  with open(annotation_file, 'r') as file:
    lines = file.readlines()
    annotations = []
    for line in lines:
      parts = line.strip().split()
      class_label = parts[0]
      x_min, y_min, x_max, y_max = map(float, parts[1:])
      annotations.append({
          'class_label': class_label,
          'x_min': x_min,
          'y_min': y_min,
          'x_max': x_max,
          'y_max': y_max
      })
    return annotations

In [15]:
# Preprocessing data
data_dir = os.path.join(os.getcwd(), 'recyclable-images-datasets')

def preprocess_data(data_dir, image_size=(224, 224)):
  images = []
  annotations = []

  image_folder = os.path.join(data_dir, 'images', 'train')
  label_folder = os.path.join(data_dir, 'labels', 'train')

  for image_filename in os.listdir(image_folder):
    if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
      image_path = os.path.join(image_folder, image_filename)
      annotation_filename = image_filename.replace('.jpg', '.txt').replace('.png', '.txt')
      annotation_path = os.path.join(label_folder, annotation_filename)

      image = tf.io.read_file(image_path)
      image = tf.image.decode_jpeg(image, channels=3)
      image = tf.image.resize(image, image_size)
      image = image / 255.0  # Normalize image

      if os.path.exists(annotation_path):
        annotation = read_annotations(annotation_path)
        images.append(image.numpy())
        annotations.append(annotation)
      else:
        print(f'Annotation file for image {image_filename} not found')

  return np.array(images), annotations

In [16]:
images, annotations = preprocess_data(data_dir)

In [17]:
# check images and annotations
print(images[0])
for i in range(10):
  print(annotations[i])

[[[0.27056822 0.36860743 0.35292116]
  [0.17833133 0.27244896 0.28645456]
  [0.22577031 0.31988794 0.35910362]
  ...
  [0.1996989  0.5485591  0.1304109 ]
  [0.32925162 0.6434772  0.19305716]
  [0.07933023 0.4224483  0.00338119]]

 [[0.19367747 0.29479793 0.2829732 ]
  [0.1520008  0.24985994 0.2632053 ]
  [0.2302521  0.32436976 0.36862746]
  ...
  [0.30458388 0.565807   0.2168885 ]
  [0.48093227 0.62242895 0.33517385]
  [0.19287176 0.45509842 0.08192892]]

 [[0.20880353 0.31860748 0.32991198]
  [0.22517009 0.3232093  0.3557023 ]
  [0.24901962 0.33921573 0.40096042]
  ...
  [0.36376947 0.5478396  0.2500437 ]
  [0.5184473  0.49403778 0.34949958]
  [0.30157393 0.475986   0.15575583]]

 ...

 [[0.48795503 0.6047618  0.70700264]
  [0.44093588 0.5851936  0.6854737 ]
  [0.4523808  0.63137245 0.7215685 ]
  ...
  [0.7579829  0.8599437  0.96582603]
  [0.74509805 0.84705883 0.9529412 ]
  [0.76498616 0.86694694 0.9728293 ]]

 [[0.48543417 0.59097636 0.69859946]
  [0.3947579  0.5354741  0.6423769 ]


In [18]:
def create_tf_dataset(images, annotations):
  def generator():
    for img, ann in zip(images, annotations):
      classes = [a['class_label'] for a in ann]
      boundary_boxes = [[a['x_min'], a['y_min'], a['x_max'], a['y_max']] for a in ann]
      yield img, (np.array(classes, dtype=np.int32), np.array(boundary_boxes, dtype=np.float32))

  dataset = tf.data.Dataset.from_generator(
      generator,
      output_signature=(
          tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
          (
              tf.TensorSpec(shape=(None,), dtype=tf.int32),
              tf.TensorSpec(shape=(None, 4), dtype=tf.float32)
          )
      )
  )

  return dataset

In [20]:
train_dataset = create_tf_dataset(images, annotations)

In [21]:
for image, (classes, boxes) in train_dataset.take(1):
    print("Image shape:", image.shape)
    print("Classes:", classes)
    print("Boxes:", boxes)

Image shape: (224, 224, 3)
Classes: tf.Tensor([6], shape=(1,), dtype=int32)
Boxes: tf.Tensor([[0.55625  0.485156 0.125    0.082812]], shape=(1, 4), dtype=float32)
